In [ ]:
# coding=utf-8
import Putil.PutilEnvSet as penv
import Putil.base.logger as plog
import inspect
import torch

In [ ]:
import Putil.base.logger as plog
plog.PutilLogConfig.config_log_level(stream=plog.DEBUG)
plog.PutilLogConfig.config_format(plog.FormatRecommend)
plog.PutilLogConfig.config_handler(plog.stream_method)

In [ ]:
Logger = plog.PutilLogConfig('Logger').logger()
Logger.setLevel(plog.DEBUG)
BaseDebugMethodLogger = Logger.getChild('BaseDebugMethod')
BaseDebugMethodLogger.setLevel(plog.DEBUG)

In [ ]:
class BaseDebugMethod:
    def __init__(self, input_size=True):
        self._include_input_size = input_size
        self._input_size_message = None
        self._logger = None
        pass
    
    def set_logger(self, logger):
        self._logger = logger
        pass
    
    def debug_message(self, message, module, input_data, logger=None):
        current_frame = inspect.currentframe()
        self._logger = self._logger if logger is None else logger
        self._logger.debug('{2} | call in {0}\n input size: {1} --> {3}'.format(current_frame.f_back.f_lineno, self.__input_size_message(input_data), message, module.named_buffers))
        return module(input_data)
        pass
    
    def functional_debug_message(self, message, functional, argv, logger=None):
        current_frame = inspect.currentframe()
        self._logger = self._logger if logger is None else logger
        self._logger.debug('{2} | call in {0}\n input size: {1} --> {3}'.format(current_frame.f_back.f_lineno, self.__input_size_message(input_data), message, module.named_buffers))
        pass
    
    def __input_size_message(self, input_data):
        self._input_size_message = ''
        def __collector(_input_data):
            if isinstance(_input_data, torch.Tensor):
                self._input_size_message = '{0} \n {1}'.format(self._input_size_message, _input_data.size())
                pass
            elif hasattr(_input_data, '__iter__'):
                if isinstance(_input_data, dict):
                    pass
                else:
                    for _input_data in input_data:
                        __collector(_input_data)
                        pass
                    pass
                pass
            pass
        __collector(input_data)
        return self._input_size_message
        pass
    pass

In [ ]:
def test():
    import torch
    class A(torch.nn.Module):
        def __init__(self):
            super(A, self).__init__()
            self._conv1 = torch.nn.Conv2d(3, 3, 3, 3, 1)
            self._conv2 = torch.nn.Conv2d(3, 3, 3, 3, 1)
            pass
        
        def forward(self, input_data):
            self._conv1(input_data[0])
            return self._conv2(input_data[1])
            pass
        pass
    a = A()
    b = BaseDebugMethod()
    import numpy as np
    c = torch.from_numpy(np.zeros(shape=[1, 3, 70, 40], dtype=np.float32))
    b.debug_message('asdasd', a, [c, c], logger=BaseDebugMethodLogger).size()
    
    # b.functional_debug_message('adada', torch.cat, ([c, c], dim=1), logger=BaseDebugMethodLogger).size()

In [ ]:
# test()